In [2]:
# note: this file exists to minify the large set of glassdoor data, reducing to one highly reviewed example of the job per job title.
# these will be turned into example recomendations.

# as an extension, multiple could be offered.  For now, we just give one example per
# import pandas, np, importer, json
import pandas as pd
import numpy as np

In [3]:
# import csv from downloads
import csv

# read in csv file "glassdoor_reviews.csv" from ~/Downloads
df = pd.read_csv("~/Downloads/glassdoor_reviews.csv")

In [15]:
# remove all companies with less than 50 reviews
df_over_50 = df.groupby('firm').filter(lambda x: len(x) > 50)
# add average rating column
df_over_50['average_firm_rating'] = df_over_50.groupby('firm')['overall_rating'].transform('mean')
df_over_50['review_count_by_firm_role'] = df_over_50.groupby(['firm', 'job_title'])['overall_rating'].transform('count')
# remove all job_title is not in ["Anonymous Employee", None]
df_over_50 = df_over_50[df_over_50.job_title != "Anonymous Employee"]
df_over_50 = df_over_50[df_over_50.job_title != " Anonymous Employee"]
df_over_50 = df_over_50[df_over_50.job_title != ""]
df_over_50 = df_over_50[df_over_50.job_title != " "]
# filter all reviews with rating < 4
df_over_50 = df_over_50[df_over_50.overall_rating == 5]
# remove all columns except firm, job_title, review_title, review_body, average_firm_rating
# df_over_50 = df_over_50[['firm', 'job_title', 'review_title', 'review_body', 'average_firm_rating']]
# select one review per job_title, selecting the one with the highest average_firm_rating, highest date_review
df_over_50 = df_over_50.sort_values(['job_title', 'review_count_by_firm_role', 'date_review'], ascending=[True, False, False])
df_over_50 = df_over_50.drop_duplicates(subset=['job_title'], keep='first')

In [16]:
df_over_50

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons,average_firm_rating,review_count_by_firm_role
122964,CBRE,2009-10-23,Administrative,Former Employee,"Atlanta, GA",5,4.0,NaN,NaN,2.0,4.0,4.0,v,v,o,CBRE CARES,The HR team is awesome and very supportive of ...,"For administrative personnel, not alot of oppo...",3.660349,1
807143,University-of-Sheffield,2008-09-02,Administrator,Current Employee,"Sheffield, England, England",5,5.0,NaN,NaN,3.0,4.0,5.0,v,v,o,"Nice place to work, Nice place to Live around,...","Job security, manageable stress level, benefit...",Less pay than industry. Recent system implemen...,4.370656,1
774789,Thomson-Reuters,2008-06-13,Analyst,Current Employee,"São Paulo, São Paulo, São Paulo",5,4.0,NaN,NaN,4.0,4.0,5.0,v,v,o,Very Nice place to work.,Great work environment and professional oportu...,Distance between different working groups.,3.578771,1
408420,J-P-Morgan,2008-06-11,Applications Architect,Current Employee,"Columbus, OH",5,5.0,NaN,NaN,4.0,4.0,3.0,v,v,o,"JPMorgan Chase: Once your in, your nuts if you...",The work/life balance at JPMorgan Chase is exc...,"It's a very large company, so there are many b...",3.712443,3
24112,Aon,2009-01-19,Assistant Director,Current Employee,NaN,5,4.0,NaN,NaN,5.0,4.0,5.0,v,v,o,Aon is a great place to work!,There are a lot of opportunities at Aon. It i...,There seem to be a lot of cliques and people g...,3.475472,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822237,Waitrose,2014-05-11,"supermarket assistant, delivery driver","Former Employee, more than 3 years","London, England, England",5,3.0,5.0,NaN,5.0,4.0,4.0,v,v,v,Fantastic company!,Lots of great benefits\nPeople are very welcom...,The work is sometimes a little boring\nAs with...,3.943861,2
726108,SAP,2020-09-24,supplier account manager,"Current Employee, more than 3 years",Singapore,5,5.0,5.0,NaN,5.0,5.0,5.0,v,v,v,Best Place to Work,"Flexibility, Employee Friendly Policies, Innov...",Processes could be a little more streamlined,4.254532,2
472032,Kingsley-Healthcare,2021-02-23,supportworker,"Current Employee, more than 1 year","Lowestoft, England, England",5,4.0,5.0,5.0,5.0,5.0,5.0,v,o,v,Kingsley experience,During the past year with the difficult times ...,There's been some challenging moments,4.500000,1
62551,BNP-Paribas,2015-10-22,uniteller,"Former Employee, more than 1 year",Cairo,5,2.0,5.0,NaN,4.0,5.0,5.0,v,r,r,Uniteller,Very professional in doing their job,You can not balance between your work and your...,3.508651,1


In [35]:
# match the string to the closest job title in the job title list
# return the full row from df_over_50
def match_job_title(job_title):
    # lookup in the df_over_50 dataframe for the closest job title, using fuzzy matching
    df_over_50_filtered = df_over_50[df_over_50['review_count_by_firm_role'] > 50]
    matching = df_over_50_filtered[df_over_50_filtered['job_title'].str.contains(job_title, case=False)].sort_values('average_firm_rating', ascending=False)
    # if there is no match, return None
    if matching.empty:
        return {
            "firm": "No Match",
            "date_review": "No Match",
            "job_title": "No Match",
            "average_firm_rating": "No Match",
            "overall_rating": "No Match",
            "headline": "No Match",
            "pros": "No Match",
            "cons": "No Match",
        }
    else:
        first = matching.iloc[0]
        return {
            "firm": first['firm'],
            "date_review": first['date_review'],
            "job_title": first['job_title'],
            "average_firm_rating": first['average_firm_rating'],
            "overall_rating": first['overall_rating'],
            "headline": first['headline'],
            "pros": first['pros'],
            "cons": first['cons'],
        }

In [36]:
match_job_title("engineer")

firm                                                     Google
date_review                                          2021-06-04
job_title                                     Software Engineer
current                                        Current Employee
location                                                    NaN
overall_rating                                                5
work_life_balance                                           4.0
culture_values                                              4.0
diversity_inclusion                                         4.0
career_opp                                                  4.0
comp_benefits                                               4.0
senior_mgmt                                                 4.0
recommend                                                     v
ceo_approv                                                    v
outlook                                                       v
headline                                

{'firm': 'Google',
 'date_review': '2021-06-04',
 'job_title': ' Software Engineer',
 'average_firm_rating': 4.392997811816192,
 'overall_rating': 5,
 'headline': 'Good working experience',
 'pros': 'It has a good working environment.',
 'cons': 'there is no obvious cons overall.'}

In [40]:
# export the dataframe to a csv file
df_over_50.to_csv('glassdoor_top_firm_and_review.csv', index=False)

In [41]:
# import csv from data1/glassdoor_top_firm_and_review.csv

true_df = pd.read_csv("data1/glassdoor_top_firm_and_review.csv")